In [15]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
import pandas as pd
import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split

In [60]:
from sklearn.metrics import roc_auc_score
import random
import os

## 构建数据

In [5]:
iris = datasets.load_iris()
X = iris['data']
y = iris['target']
X = X[y!=2]
y = y[y!=2]
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [6]:
train_df = pd.DataFrame(X_train, columns=['a', 'b', 'c', 'd'])
test_df = pd.DataFrame(X_test, columns=['a', 'b', 'c', 'd'])
train_df['label'] = pd.Series(y_train)
test_df['label'] = pd.Series(y_test)

In [7]:
train_df.head()

,a,b,c,d,label
0,6.7,3.0,5.0,1.7,1
1,5.9,3.2,4.8,1.8,1
2,4.9,2.4,3.3,1.0,1
3,6.1,2.8,4.7,1.2,1
4,6.4,3.2,4.5,1.5,1


In [8]:
test_df.head()

,a,b,c,d,label
0,4.9,3.1,1.5,0.1,0
1,5.3,3.7,1.5,0.2,0
2,6.6,2.9,4.6,1.3,1
3,6.0,3.4,4.5,1.6,1
4,5.2,3.5,1.5,0.2,0


In [12]:
device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')
batch_size = 10
num_workers = 4
lr = 1e-4
epochs = 20

In [10]:
class MyDataset(Dataset):
    def __init__(self, df):
        self.df = df
        self.X = df.loc[:, ['a', 'b', 'c', 'd']].values
        self.y = df.loc[:, 'label'].values
    def __len__(self):
        return len(self.X)
    def __getitem__(self, idx):
        data_x = self.X[idx]
        data_y = self.y[idx]
        return data_x, data_y


In [33]:
train_data = MyDataset(train_df)
test_data = MyDataset(test_df)

train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True, num_workers=num_workers, drop_last=True)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False, num_workers=num_workers)

In [14]:
data_x, data_y = next(iter(train_loader))
print(data_x)
print(data_y)

tensor([[5.2000, 4.1000, 1.5000, 0.1000],
        [5.4000, 3.9000, 1.3000, 0.4000],
        [4.8000, 3.1000, 1.6000, 0.2000],
        [5.0000, 2.3000, 3.3000, 1.0000],
        [6.0000, 2.2000, 4.0000, 1.0000],
        [5.1000, 3.7000, 1.5000, 0.4000],
        [5.8000, 2.7000, 3.9000, 1.2000],
        [4.5000, 2.3000, 1.3000, 0.3000],
        [5.4000, 3.9000, 1.7000, 0.4000],
        [6.1000, 2.9000, 4.7000, 1.4000]], dtype=torch.float64)
tensor([0, 0, 0, 1, 1, 0, 1, 0, 0, 1])


In [74]:
# def setup_seed(seed):
#     torch.manual_seed(seed)
#     torch.cuda.manual_seed_all(seed)
#     np.random.seed(seed)
#     random.seed(seed)
#     torch.backends.cudnn.deterministic = True
# setup_seed(2020)
def seed_torch(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.enabled = False
seed_torch(2020)

## 搭建模型

In [75]:
class FMLayer(nn.Module):
    def __init__(self, n=10, k=5):
        """
        param n:特征维度
        param k:隐向量维度
        """
        super(FMLayer, self).__init__()
        self.dtype = torch.float
        self.n = n
        self.k = k
        self.linear = nn.Linear(self.n, 1)  #前两项线性层
        self.v = nn.Parameter(torch.randn(self.n, self.k)) #交互矩阵
        nn.init.uniform_(self.v, -0.1, 0.1)
    def fm_layer(self, x):
        #x的维度是(batch_size, n)
        linear_part = self.linear(x)
        #根据上面的公式计算二阶部分
        inter_part1 = torch.mm(x, self.v)
        inter_part2 = torch.mm(torch.pow(x, 2), torch.pow(self.v, 2))
        inter = 0.5 * torch.sum(torch.sub(torch.pow(inter_part1, 2), inter_part2), 1, keepdim=True)
        output = linear_part + inter
        output = torch.sigmoid(output)
        return output
    def forward(self, x):
        return self.fm_layer(x)
    def fit(self, data, optimizer, epochs=100):
        #训练模型并输出测试集每一轮的loss
        criterion = F.binary_cross_entropy
        for epoch in range(epochs):
            for t, (batch_x, batch_y) in enumerate(data):
                batch_x = batch_x.float().to(device)
                batch_y = batch_y.float().to(device)
                total = self.forward(batch_x)
                loss = criterion(total, batch_y)
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
            loader_test = DataLoader(test_data, batch_size=10, shuffle=False)
            
            r = self.test(loader_test)
            print('Epoch %d, loss=%.4f' % (epoch, r))
    def test(self, data):
        #测试集测试
        criterion = F.binary_cross_entropy
        all_loss = 0
        gt_labels = []
        pred_labels = []
        i = 0
        with torch.no_grad():
            for t, (batch_x, batch_y) in enumerate(data):
                batch_x = batch_x.float().to(device)
                batch_y = batch_y.float().to(device)
                pred = self.forward(batch_x)
                gt_label = batch_y.cpu().data.numpy()
                pred_proba = pred.cpu().data.numpy()
                gt_labels.append(gt_label)
                pred_labels.append(pred_proba)
                loss = criterion(pred, batch_y)
                all_loss += loss.item()
                i += 1
        gt_labels, pred_labels = np.concatenate(gt_labels), np.concatenate(pred_labels)
        pred_labels = pred_labels.reshape(len(pred_labels),)
        auc = roc_auc_score(gt_labels, pred_labels)
        print('auc:', auc, 'gt_lables:', gt_labels.shape, 'pred_labels:', pred_labels.shape)
        return all_loss / i

## 使用数据训练和测试模型

In [76]:
fm = FMLayer(n=4, k=5)
fm = fm.to(device)
optimizer = optim.Adam(fm.parameters(), lr=lr, weight_decay=0.0)
fm.fit(train_loader, optimizer, epochs=50)

auc: 0.9358974358974359 gt_lables: (25,) pred_labels: (25,)
Epoch 0, loss=0.7545


/home/guoyuhao/.local/lib/python3.6/site-packages/ipykernel_launcher.py:34: UserWarning: Using a target size (torch.Size([10])) that is different to the input size (torch.Size([10, 1])) is deprecated. Please ensure they have the same size.
/home/guoyuhao/.local/lib/python3.6/site-packages/ipykernel_launcher.py:58: UserWarning: Using a target size (torch.Size([10])) that is different to the input size (torch.Size([10, 1])) is deprecated. Please ensure they have the same size.
/home/guoyuhao/.local/lib/python3.6/site-packages/ipykernel_launcher.py:58: UserWarning: Using a target size (torch.Size([5])) that is different to the input size (torch.Size([5, 1])) is deprecated. Please ensure they have the same size.


auc: 0.9487179487179487 gt_lables: (25,) pred_labels: (25,)
Epoch 1, loss=0.7467


/home/guoyuhao/.local/lib/python3.6/site-packages/ipykernel_launcher.py:34: UserWarning: Using a target size (torch.Size([10])) that is different to the input size (torch.Size([10, 1])) is deprecated. Please ensure they have the same size.
/home/guoyuhao/.local/lib/python3.6/site-packages/ipykernel_launcher.py:58: UserWarning: Using a target size (torch.Size([10])) that is different to the input size (torch.Size([10, 1])) is deprecated. Please ensure they have the same size.
/home/guoyuhao/.local/lib/python3.6/site-packages/ipykernel_launcher.py:58: UserWarning: Using a target size (torch.Size([5])) that is different to the input size (torch.Size([5, 1])) is deprecated. Please ensure they have the same size.


auc: 0.9615384615384616 gt_lables: (25,) pred_labels: (25,)
Epoch 2, loss=0.7387


...

/home/guoyuhao/.local/lib/python3.6/site-packages/ipykernel_launcher.py:34: UserWarning: Using a target size (torch.Size([10])) that is different to the input size (torch.Size([10, 1])) is deprecated. Please ensure they have the same size.
/home/guoyuhao/.local/lib/python3.6/site-packages/ipykernel_launcher.py:58: UserWarning: Using a target size (torch.Size([10])) that is different to the input size (torch.Size([10, 1])) is deprecated. Please ensure they have the same size.
/home/guoyuhao/.local/lib/python3.6/site-packages/ipykernel_launcher.py:58: UserWarning: Using a target size (torch.Size([5])) that is different to the input size (torch.Size([5, 1])) is deprecated. Please ensure they have the same size.


auc: 1.0 gt_lables: (25,) pred_labels: (25,)
Epoch 48, loss=0.5175


/home/guoyuhao/.local/lib/python3.6/site-packages/ipykernel_launcher.py:34: UserWarning: Using a target size (torch.Size([10])) that is different to the input size (torch.Size([10, 1])) is deprecated. Please ensure they have the same size.
/home/guoyuhao/.local/lib/python3.6/site-packages/ipykernel_launcher.py:58: UserWarning: Using a target size (torch.Size([10])) that is different to the input size (torch.Size([10, 1])) is deprecated. Please ensure they have the same size.
/home/guoyuhao/.local/lib/python3.6/site-packages/ipykernel_launcher.py:58: UserWarning: Using a target size (torch.Size([5])) that is different to the input size (torch.Size([5, 1])) is deprecated. Please ensure they have the same size.


auc: 1.0 gt_lables: (25,) pred_labels: (25,)
Epoch 49, loss=0.5161


/home/guoyuhao/.local/lib/python3.6/site-packages/ipykernel_launcher.py:34: UserWarning: Using a target size (torch.Size([10])) that is different to the input size (torch.Size([10, 1])) is deprecated. Please ensure they have the same size.
/home/guoyuhao/.local/lib/python3.6/site-packages/ipykernel_launcher.py:58: UserWarning: Using a target size (torch.Size([10])) that is different to the input size (torch.Size([10, 1])) is deprecated. Please ensure they have the same size.
/home/guoyuhao/.local/lib/python3.6/site-packages/ipykernel_launcher.py:58: UserWarning: Using a target size (torch.Size([5])) that is different to the input size (torch.Size([5, 1])) is deprecated. Please ensure they have the same size.


In [77]:
fm.test(test_loader)

auc: 1.0 gt_lables: (25,) pred_labels: (25,)


/home/guoyuhao/.local/lib/python3.6/site-packages/ipykernel_launcher.py:58: UserWarning: Using a target size (torch.Size([10])) that is different to the input size (torch.Size([10, 1])) is deprecated. Please ensure they have the same size.
/home/guoyuhao/.local/lib/python3.6/site-packages/ipykernel_launcher.py:58: UserWarning: Using a target size (torch.Size([5])) that is different to the input size (torch.Size([5, 1])) is deprecated. Please ensure they have the same size.


0.5160911977291107